##### Obtendo valores de campo (G) quando a M/Ms é zero (durante a "descida" e "subida" da curva de histerese) para cada ângulo
A coercividade Hc é obtido atraves da média em módulo dos dois pontos de G quando M/Ms é igual a zero. Em uma curva de histerese teremos para cada ângulo um valor de G com M/Ms igual a zero durante a "descida e outro valor de G com M/Ms igual a zero durante a "subida".

In [ ]:
#######################################
## CARREANDO BIBLIOTECAS NECESSÁRIAS ##
#######################################

import numpy as np #manipulação de arrays e aplicações matemáticas #type: ignore
import matplotlib.pyplot as plt #plot de figuras #type: ignore
import os #manipulação de diretórios/arquivos
from caminhosDiretorios import dirDadosDesteDispositivo as dirDados #variavel que guarda o caminho do diretório dos arquivos de medida
from funcoesDeMatrizENormalizacao import extrair_angulo

"""
CHAMANDO E RECONHECENDO DADOS
"""

# Lista todos os arquivos na pasta 

caminho_pasta = dirDados+"dadosNormalizados"

arquivos = os.listdir(caminho_pasta)

# Filtra apenas os arquivos .txt
arquivos_txt = [arquivo for arquivo in arquivos if arquivo.endswith('.txt')]

# Ordena os arquivos pelo ângulo através do método "sort()"
arquivos_txt.sort(key=extrair_angulo)

##############################################################################################
## OBTENDO VALORES DE G (na "descida" e na "subida") quando o M/Ms é zero - para cada ângulo##
##############################################################################################

# Lista para armazenar os valores de G quando o M/Ms é zero para cada ângulo
valores_G = []

# Obtendo os valores de G quando o M/Ms é zero - tanto na "descida" quanto na "subida" (para cada ângulo)
for arquivo_txt in arquivos_txt:
    # Extrai o ângulo do nome do arquivo
    angulo = extrair_angulo(arquivo_txt)

    # Caminho completo do arquivo
    caminho_arquivo = os.path.join(caminho_pasta, arquivo_txt)
    
    # Carrega os dados do arquivo, ignorando 12 linhas
    dados = np.loadtxt(caminho_arquivo)

    # Extrai os dados de campo e magnetização
    campo = dados[:, 0]
    magnetizacao = dados[:, 1]
    
    # Normalizando apenas a magnetização
    magnetizacao_normalizada = (2 * magnetizacao - (magnetizacao.max() + magnetizacao.min())) / (magnetizacao.max() - magnetizacao.min())

    # Encontrando os índices onde o M/Ms é zero durante a descida e durante a subida
    indices_zero_descida = np.where(np.diff(np.signbit(magnetizacao_normalizada)))[0][0]
    indices_zero_subida = np.where(np.diff(np.signbit(magnetizacao_normalizada)))[0][-1] + 1
    
    # Calculando os valores de G quando o M/Ms é zero durante a descida e durante a subida
    valor_G_descida = campo[indices_zero_descida]
    valor_G_subida = campo[indices_zero_subida]

    # Armazenando os valores para este ângulo
    valores_G.append((angulo, valor_G_descida, valor_G_subida))

# Exibindo os valores de G quando o M/Ms é zero para cada ângulo
print("Valores de G quando o M/Ms é zero para cada ângulo:")
for angulo, valor_G_descida, valor_G_subida in valores_G:
    print(f'Ângulo {angulo}° (Descida): {valor_G_descida:.2f}')
    print(f'Ângulo {angulo}° (Subida): {valor_G_subida:.2f}')
    
#########################################################
## OBTENDO OS VALORES DE COERCIVIDADE PARA CADA ÂNGULO ##
#########################################################

# Lista para armazenar os valores de coercividade para cada ângulo
valores_coercividade = []

# Calculando a média em módulo dos valores de M/Ms para cada ângulo
for angulo, valor_G_descida, valor_G_subida in valores_G:
    # Calcula a média em módulo dos valores de descida e subida
    media_descida_subida = (np.abs(valor_G_descida) + np.abs(valor_G_subida)) / 2
    
    # Armazena o valor de magnetização de remanência para este ângulo
    valores_coercividade.append((angulo, media_descida_subida))

# Exibindo os valores de magnetização de remanência para cada ângulo
print("\nValores de coercividade para cada ângulo:")
for angulo, valor_coercividade in valores_coercividade:
    print(f'Ângulo {angulo}°: Hc = {valor_coercividade:.2f}')


In [ ]:
###################################################
## PLOTANDO A COERCIVIDADE (Hc) PARA CADA ÂNGULO ##
###################################################

import matplotlib.pyplot as plt #type: ignore

# Extrai os ângulos e os valores de coercividade
angulos = [angulo for angulo, _ in valores_coercividade]
valores = [valor for _, valor in valores_coercividade]

# Cria o gráfico de linha
plt.figure(figsize=(10, 6))
plt.plot(angulos, valores, marker='o', color='skyblue', linestyle='-')

# Define os valores de ângulo no eixo x variando de 10 em 10
plt.xticks(range(min(angulos), max(angulos)+1, 10,))

# Adiciona rótulos e título
plt.xlabel('Ângulo (°)', fontsize=14)
plt.ylabel('Coercividade (Hc)', fontsize=14)
plt.title('Coercividade (Hc) para Cada Ângulo', fontsize=16)

# Adiciona grade
plt.grid(True)

# Exibe o gráfico
plt.show()


In [ ]:
############################################
## DESLOCANDO OS DADOS 10° PARA A DIREITA ## (Fazer apenas se necessário)
############################################

# Deslocando os dados 10° para a direita
valores_coercividade_deslocados = [(angulo + 10 if angulo + 10 < 360 else angulo + 10 - 360, valor) for angulo, valor in valores_coercividade]

#########################
## DUPLICANDO OS DADOS ##
#########################

# Criando um dicionário para armazenar valores únicos
valores_coercividade_dict = {angulo: valor for angulo, valor in valores_coercividade_deslocados}

# Atualizando novamente os valores de 0° e 360° após duplicação
if 0 not in valores_coercividade_dict:
    valores_coercividade_dict[0] = valores_coercividade_dict.get(350, valores_coercividade_dict.get(10))
if 360 not in valores_coercividade_dict:
    valores_coercividade_dict[360] = valores_coercividade_dict.get(0, valores_coercividade_dict.get(350))

# Duplicando os dados para 0-360 graus, evitando duplicações para ângulo igual
for angulo, valores_coercividade in list(valores_coercividade_deslocados):
    if angulo > 0 and angulo < 180:
        angulo_duplicado = 360 - angulo
        if angulo_duplicado not in valores_coercividade_dict:
            valores_coercividade_dict[angulo_duplicado] = valores_coercividade

# Ordenando os valores duplicados por ângulo
valores_coercividade_duplicados = sorted(valores_coercividade_dict.items())

# Separando ângulos e valores de remanência
angulos = [x[0] for x in valores_coercividade_duplicados]
coercividades = [x[1] for x in valores_coercividade_duplicados]

# Convertendo ângulos para radianos para plotagem polar
angulos_rad = np.radians(angulos)

########################################
## PLOTANDO OS DADOS EM GRÁFICO POLAR ##
########################################

# Plotando os resultados em coordenadas polares
plt.figure(figsize=(10, 6))
ax = plt.subplot(111, projection='polar')
ax.plot(angulos_rad, coercividades, 'o-', label='Coercividade')
ax.set_theta_direction(-1)  # Direção dos ângulos no sentido horário
ax.set_theta_offset(np.pi / 2.0)  # Define 0° no topo
ax.set_xticks(np.radians(np.arange(0, 360, 30)))
ax.set_xticklabels([f'{i}°' for i in range(0, 360, 30)])
plt.title('Anisotropia da Coercividade (Hc) para Cada Ângulo (amostra: Ta_CoFeB_Ag_x15)')
#plt.legend()
plt.grid(True)
plt.show()